## LangChain
Documentation: https://docs.langchain.com/docs/

It is a framework for developing applications powered by LLM (Large Language Models). The most powerful and differentiated applications will not only call to a language model via an API, but will also:
- Be data-aware: connect a language model to other sources of data
- Be agentic: Allow a language model to interact with its environment

There are two main value props the LangChain framework provides:
1. **Components**: LangChain provides modular abstractions for the components neccessary to work with language models. LangChain also has collections of implementations for all these abstractions. The components are designed to be easy to use, regardless of whether you are using the rest of the LangChain framework or not.
2. **Use-Case specific chains**: Chains can be thought of as assembling these components in particular ways in order to best accomplish a particular use case. These are intended to be a higher level interface through which people can easily get started with a specific use case. These chains are also designed to be customizable.

### Components
- Models
- Prompts
- Indexes
- Memory
- Chains
- Agents

We'll first go through some underlying schema abstractions, before diving into the six main components of LangChain.

#### Schema
The basic data types and schemas that are used throughout the codebase.
- **Text**: When working with the LLM, the primary interface through which you can interact with them is through text. As an over simplification, a lot of models are "text-in", "text-out". Therefore, a lot of the interface in LangChain are centered around text.
- **ChatMessages**: The primary interface through with end users interact with these is a chat interface.
  - **SystemChatMessage** - A chat message representing information that should be instructions to the AI system.
  - **HumanChatMessage** - A chat message representing information coming from a human interacting with the AI system.
  - **AIChatMessage** - A chat message representing information coming from the AI system.

#### Models
This section of the documentation deals with different types of models that are used in LangChain.
- **LLMs** - Large Language Model (LLMs) are the first type of model. These models takes text string as input and return a text string as output.
- **ChatModels** - Chat Models are the second type of models we cover. These models are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.
- **Text Embedding Models** - The third type of models we cover are text embedding models. These models take text as input and return a list of floats.

#### Prompts
The new way of programming models is through prompts. A "prompt" refers to the input to the model. This input is rarely hard coded, but rather is often constructed from multiple components. A PromptTemplate is responsible for the construction of this input. LangChain provides several classes and functions to make constructing and working with prompts easy.

It has 4 sections:
- **PromptValue** - The class representing an input to a model.
- **PromptTemplate** - The class in charge of constructing a `PromptValue`
- **Example Selectors** - Often times it is useful to include examples in prompts. These examples can be hardcoded, but it s often more powerful if they are dynamically selected.
- **Output Parsers** - Language Model (and Chat Models) output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output parsers are responsible for:
  - Instructing the model how output should be formatted.
  - Parsing output into the desired formatting (including the retries if necessary)

#### Indexes
Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents, different types of indexes, and then examples for using those indexes in chains.

The most common way that indexes are used in chains is in a "retrieval" step. This step refers to taking a user's query and returning the most relevant documents.

Most of the time when we talk about indexes and retrieval we are talking about indexing and retrieving unstructured data (like text documents). For interacting with structured data (SQL tables, etc) or APIs, please see the corresponding use case sections for links to relevant functionality. The primary index and retrieval types supported by LangChain are currently centered around vector databases, and therefore a lot of the functionality we dive deep on those topics.

- **Document Loaders** - Classes responsible for loading documents from various sources.
- **Text Splitters** - Classes responsible for splitting text into smaller chunks.
- **VectorStores** - The most common type of index. One that relies on embeddings. The most common type of index is one that creates numerical embeddings (with an Embedding Model) for each document. A vectorstore stores Documents and associated embeddings, and provides fast ways to look up relevant Documents by embeddings.
- **Retrievers** - Interface for fetching relevant documents to combine with language models. A way of storing data such that it can be queried by a language model. The only interface the object must expose is a `get_relevant_texts` method which takes in a string and returns list of documents.

#### Memory
Memory is the concept of storing and retrieving data in the process of a conversation. There are two main methods:
1. Based on input, fetch any relevant pieces of data.
2. Based on the input and output, update state accordingly.

There are two main types of memory: short term and long term.
- Short term memory generally refers to how to pass data in the context of a singular conversation (generally is previous ChatMessages or summaries of them).
- Long term memory deals with how to fetch and update information between conversations.

**Chat Message History**
The primary interface with language models at the moment in through a chat interface. The ChatMessageHistory class is responsible for remembering all previous chat interactions. These can then be passed directly back into the model, summarized in some way, or some combination.

#### Chains
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.

The most commonly used type of chain is an LLMChain, which combines a PromptTemplate, a Model, and Guardrails to take user input, format it accordingly, pass it to the model and get a response, and then validate and fix (if necessary) the model output.

- **Chain** - A chain is just an end-to-end wrapper around multiple individual components.
- **LLMChain** - A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
- **Index-related chains** - This category of chains are used for interacting with indexes. The purpose these chains is to combine your own data (stored in the indexes) with LLMs. The best example of this is question answering over your own documents. A big part of this is understanding how to pass multiple documents to the language model. There are a few different methods, or chains, for doing so. LangChain supports four of the more common ones.
  - **Stuffing** - It is the simplest method, where by you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as `StuffDocumentChain`
    - _PROS_: Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.
    - _CONS_: Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.
    - The main downside of this method is that it only works on smaller pieces of data. Once you are working with many pieces of data, this approach is no longer feasible. The next two approaches are designed to help deal with that.
  - **Map Reduce** - This method involves running an initial prompt on each chunk of data (for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). Then a different prompt is run to combine all the initial outputs. This is implemented in the LangChain as the `MapReduceDocumentsChain`.
    - _PROS_: Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.
    - _CONS_: Requires many more calls to the LLM than `StuffDocumentsChain`. Loses some information during the final combined call.
  - **Refine** - This method involves running an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document, asking the LLM to refine the output based on the new document.
    - _PROS_: Can pull in a more relevant context, and may be less lossy than `MapReduceDocumentsChain`.
    - _CONS_: Requires many more calls to the LLM than `StuffDocumentsChain`. The calls are also NOT independent, meaning they cannot be paralleled like `MapReduceDocumentsChain`. There is also some potential dependencies on the ordering of the documents.
  - **Map-Rerank** - This method involves running an initial prompt on each chunk of data, that not only tries to complete a task but also gives a score for how certain it is in its answer. The responses are then ranked according to this score, and the highest score is returned.
    - _PROS_: Similar pros as `MapReduceDocumentsChain`. Requires fewer calls, compared to `MapReduceDocumentsChain`.
    - _CONS_: Cannot combine information between documents. This means it is most useful when you expect there to be a single simple answer in a single document.
- **Prompt Selector** - One of the goals of chains in LangChain is to enable people to get started with a particular use case as quickly as possible. A big part of this is having good prompts. The problem is that a prompt that works for one model may not work as well for another model. We want to enable chains to work well for all types of models. Therefore, rather than hardcoding a default prompt to use in chains, we have the concept of a PromptSelector. This PromptSelector is responsible for choosing a default prompt depending on the model passed in.

#### Agents
Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user's input. In these types of chains, there is a `agent` which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.
- **Tools** - How language model interact with other models. A specific abstraction around a function that makes it easy for a language model to interact with it. Specificlaly, the interface of a tool has a single text input and a single text output.
- **Agents** - The language model that drives decision making. An Agent is a wrapper around a model, which takes in user input and returns a response corresponding to an “action” to take and a corresponding “action input”.
- **Toolkits** - Sets of tools that when used together can accomplish a specific task. Groups of tools that can be used/are necessary to solve a particular problem.
- **Agent Executor** - The logic for running agents with tools. An Agent Executor is an Agent and set of Tools. The agent executor is responsible for calling the agent, getting back and action and action input, calling the tool that the action references with the corresponding input, getting the output of the tool, and then passing all that information back into the Agent to get the next action it should take.